<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/Ejercicio_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 6

## Cargar la base

La siguiente base describe datos sobre los salarios, la formación y la experiencia de empleadores que realizan entrevistas de trabajo.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mohithsairamreddy/salary-data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
salary-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [ ]:
## Cargamos la base
df = pd.read_csv('/content/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


## Procesamiento de la base

Vemos que la variable `Education Level` tiene varios valores similares escritos de manera distinta.

In [ ]:
df["Education Level"].value_counts()

Bachelor's Degree    2267
Master's Degree      1573
PhD                  1368
Bachelor's            756
High School           448
Master's              288
phD                     1
Name: Education Level, dtype: int64

Para unificar estos valores, la recatogrizamos creando una nueva variable denominada `Education New`:

In [ ]:
df["Education New"]=df["Education Level"].copy()

In [ ]:
df["Education New"].loc[df["Education Level"].str.contains("Bach")==True]="Bachelor's"
df["Education New"].loc[df["Education Level"].str.contains("Mast")==True]="Master's"
df["Education New"].loc[df["Education Level"].str.contains("hD")==True]="PhD"


<ipython-input-18-42d4324270b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Education New"].loc[df["Education Level"].str.contains("Bach")==True]="Bachelor's"
<ipython-input-18-42d4324270b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Education New"].loc[df["Education Level"].str.contains("Mast")==True]="Master's"
<ipython-input-18-42d4324270b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Education New"].loc[

Verificamos que con estos cambios, las categorías quedaron unificadas

In [ ]:
df["Education New"].value_counts()

Bachelor's     3023
Master's       1861
PhD            1369
High School     448
Name: Education New, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6702 non-null   float64
 1   Gender               6702 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6702 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6699 non-null   float64
 6   Education New        6701 non-null   object 
dtypes: float64(3), object(4)
memory usage: 366.8+ KB


Borramos además los datos faltantes de la base

In [ ]:
df=df.dropna()

Verificamos que ahora no quedan datos faltantes

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6698 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6698 non-null   float64
 1   Gender               6698 non-null   object 
 2   Education Level      6698 non-null   object 
 3   Job Title            6698 non-null   object 
 4   Years of Experience  6698 non-null   float64
 5   Salary               6698 non-null   float64
 6   Education New        6698 non-null   object 
dtypes: float64(3), object(4)
memory usage: 418.6+ KB


# Preguntas

## Pregunta 1

Contar la cantidad total de individuos y la cantidad de personas de cada género en la base. Con estos datos, calcular el porcentaje de cada género en la base.

*Ayuda:* el atributo `.shape` devuelve la cantidad de filas y columnas de una base. Por lo tanto, tomando `df.shape[0]` se calcula la cantidad de filas de la base `df` y por lo tanto, la cantidad de individuos.
Para contar la cantidad de mujeres, por ejemplo, se puede recorrer todas las filas de "Gender" viendo en cada instancia si el valor coincide con "Female". Cuando se suman esos valores, se obtiene la cantidad de mujeres: `np.sum(df["Gender"]=="Female")`


## Pregunta 2

Notemos que el porcentaje identificado como `Other` en la base es pequeño y no es un número que permita obtener conclusiones estadísticas pertinentes.

El porcentaje de hombres y mujeres parece equilibrado, sin embargo, calcular un intervalo de confianza para la proporción de mujeres. ¿Diría que podría considerarse del 50% el porcentaje de mujeres?

*Ayuda:* Utilizar la función provista y los datos del ejercicio anterior. En caso de necesitarlo, utilice el "help" para entender cómo funciona la función. Recordar que el comando devuelve proporciones (entre 0 y 1), no porcentajes (entre 0 y 100).

In [ ]:
from statsmodels.stats.proportion import proportion_confint


In [ ]:
help(proportion_confint)

## Pregunta 3

Calcular un intervalo de confianza para el salario medio y comparar los salarios medios entre hombres y mujeres con la misma metodología, a partir de los datos de esta base.

*Ayuda:* Se están evaluando medias, no porcentajes, por lo que se usa la función provista. La diferencia entre ambos intervalos reside en la cantidad de conjuntos de datos que se pasan como argumento. Para identificar, por ejemplo, los datos de salarios sólo de las mujeres, se puede utilizar el siguiente comando: `df.Salary[df["Gender"]=="Female"]`

In [ ]:
from statsmodels.stats.weightstats import zconfint

In [ ]:
help(zconfint)

## Pregunta 4

Utilizar el comando `relplot` del paquete `seaborn` para ver cómo se distribuyen los salarios según el nivel educativo (utilizar la variable `Education New` para evitar un exceso de paneles), haciendo un scatterplot por cada categoría.

*Ayuda:* El comando `relplot` puede hacer gráficos de línea (lineplot) o de dispersión (scatterplot) separando automáticamente por los valores de alguna variable categórica. Por ejemplo, para este caso, el objetivo es utilizar la base (`data=df`) para hacer un scatterplot (`kind="scatter"`) donde se visualice la relación entre `x="Gender"` e `y="Salary"`, separando en columnas por `col="Education New"`.

**Comentario:** Notar lo que sucede con el nivel "Other" de la variable "Gender". Hay mayor representación en los que tienen el secundario completo. Esto podría estar asociado a la edad y una conclusión inicial (pero incompleta) podría ser que las personas que se identifican con género no binario crecieron con el tiempo, ya que asumimos que son personas más jóvenes al no haber llegado a un mayor nivel educativo. Sin embargo, esta conclusión inicial omite que con el tiempo las personas que no se identifican con un género específico durante años sintieron la necesidad de reprimirlo y que con las discusiones que se llevan a cabo hoy ya no sienten esa necesidad.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
help(sns.relplot)

## Pregunta 5

Comparar con intervalos de confianza el salario medio de hombres y mujeres a través de todos los niveles educativos, a partir de los datos de la base. Es decir, seleccionar los salarios de los hombres que tienen el secundario completo y compararlo con los salarios de las mujeres que tienen el secundario completo, y así sucesivamente con los otros niveles educativos. Serían 4 intervalos de confianza.

*Ayuda:* Es similar al ejercicio 3, aunque hay que hacer una selección extra para cada intervalo. Para seleccionar, por ejemplo, las mujeres que tienen el secundario completo, tendrían que usar el comando
`df.Salary[(df["Gender"]=="Female") & (df["Education New"]=="High School")]`

In [ ]:
from statsmodels.stats.weightstats as smsw
df=df[np.isnan(df["Salary"])==False]
dfM=df[df["Gender"]=="Male"]
dfM=dfM[np.isnan(dfM["Salary"])==False]
dfF=df[df["Gender"]=="Female"]
dfF=dfF[np.isnan(dfF["Salary"])==False]
print(smsw.zconfint(df["Salary"],ddof=0))
print(smsw.zconfint(dfM["Salary"],dfF["Salary"],ddof=0))
print(smsw.zconfint(dfM["Salary"][dfM["Education New"]=="Bachelor's"],dfF["Salary"][dfF["Education New"]=="Bachelor's"],ddof=0))
print(smsw.zconfint(dfM["Salary"][dfM["Education New"]=="Master's"],dfF["Salary"][dfF["Education New"]=="Master's"],ddof=0))
print(smsw.zconfint(dfM["Salary"][dfM["Education New"]=="PhD"],dfF["Salary"][dfF["Education New"]=="PhD"],ddof=0))
print(smsw.zconfint(dfM["Salary"][dfM["Education New"]=="High School"],dfF["Salary"][dfF["Education New"]=="High School"],ddof=0))


(114063.01169422595, 116590.91784749669)
(10977.787363969763, 16023.957121256552)
(6612.063783349674, 13001.034178420581)
(13712.64162240878, 21019.458918054916)
(4688.2854173788255, 12201.271944147997)
(5588.686593235742, 11660.720491937089)
